# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [1]:
client_key = "o5At944dswzvw3VZbSVONmeff"
client_secret = "NPTZvV1hRiKO7chMjG4uCEpEmmem3o2Z1Gvuoj24M0He1dU52v"

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAH2%2BewEAAAAAfB%2BOlYpPqVHnaujOcNagh7DzyOI%3DnnSgWSjVFWXHruxlmVfYUFvJuBK8DAHEuZbSn7OJ8M7rgikqFU'}

In [6]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [8]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
climate_tweets = response.json()


In [10]:
len(climate_tweets['data'])

100

In [11]:
tweets +=climate_tweets['data']


In [12]:

 ids = [tw['id'] for tw in climate_tweets['data']]
    

In [13]:
min_id = min(ids)

In [15]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [14]:
 parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'until_id' : min_id
                 }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['statuses']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [15]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [16]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [28]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1315775267831382018',
  'conversation_id': '1526375358383017985',
  'created_at': '2022-05-17T23:23:01.000Z',
  'entities': {'mentions': [{'end': 16,
                             'id': '1497960029374201858',
                             'start': 0,
                             'username': 'GreenJDandWords'},
                            {'end': 31,
                             'id': '1004132506805854208',
                             'start': 17,
                             'username': 'TiemannAmelia'}]},
  'id': '1526704873995046912',
  'in_reply_to_user_id': '1315775267831382018',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 0},
  'referenced_tweets': [{'id': '1526704632960909312', 'type': 'replied_to'}],
  'reply_settings': 'everyone',
  'source': 'Twitter Web App',
  'text': '@GreenJDandWords @TiemannAmelia For myself, and most of the nuclear 

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [17]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @parashotz: Every climate change prediction since the 70’s hasn’t come true. https://t.co/yWElcdljM4

RT @AlisonBoxxer: Antifa #LaurenBoebertIsSoDumb Kyle Rittenhouse climate change Woke Capitalism America first House Republicans Major Biden…

RT @luckytran: Scientists have been warning us for decades that prolonged extreme heatwaves and out of control disease outbreaks will becom…

RT @WhipRealer: It’s a farce. Always has been. Always will be. Climate change generates dollars. That’s what it’s for. https://t.co/6xAEEDT…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [18]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['conversation_id', 'entities', 'created_at', 'public_metrics', 'id', 'author_id', 'lang', 'text', 'source', 'reply_settings', 'referenced_tweets'])
11


In [19]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [33]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here
        
        
        yield modified_tweet       
            
            

In [20]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = modified_tweet['referenced_tweets'][0]['id']
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [21]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'conversation_id': '1548769297807429632',
  'entities': {'mentions': [{'start': 3,
     'end': 13,
     'username': 'parashotz',
     'id': '542153653'}],
   'urls': [{'start': 80,
     'end': 103,
     'url': 'https://t.co/yWElcdljM4',
     'expanded_url': 'https://twitter.com/parashotz/status/1548662364794388481/video/1',
     'display_url': 'pic.twitter.com/yWElcdljM4',
     'media_key': '7_1548662278370824192'}]},
  'created_at': '2022-07-17T20:39:09.000Z',
  'public_metrics': {'retweet_count': 1110,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'id': '1548769297807429632',
  'lang': 'en',
  'text': 'RT @parashotz: Every climate change prediction since the 70’s hasn’t come true. https://t.co/yWElcdljM4',
  'source': 'Twitter for iPhone',
  'reply_settings': 'everyone',
  'referenced_tweets': [{'type': 'retweeted', 'id': '1548662364794388481'}],
  'user_id': '1351610562938073092',
  'referenced_tweets_ids': '1548662364794388481',
  'retweet_count': 1110},
 {'con

In [22]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [23]:
climate_feed_df

,conversation_id,entities,created_at,public_metrics,id,lang,text,source,reply_settings,referenced_tweets,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id
0,1548769297807429632,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",2022-07-17T20:39:09.000Z,"{'retweet_count': 1110, 'reply_count': 0, 'lik...",1548769297807429632,en,RT @parashotz: Every climate change prediction...,Twitter for iPhone,everyone,"[{'type': 'retweeted', 'id': '1548662364794388...",1351610562938073092,1548662364794388481,1110,NaN
1,1548769295534014466,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",2022-07-17T20:39:09.000Z,"{'retweet_count': 284, 'reply_count': 0, 'like...",1548769295534014466,en,RT @AlisonBoxxer: Antifa #LaurenBoebertIsSoDum...,Twitter Web App,everyone,"[{'type': 'retweeted', 'id': '1548714522143326...",1000152196061782016,1548714522143326216,284,NaN
2,1548769294514884610,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",2022-07-17T20:39:09.000Z,"{'retweet_count': 3773, 'reply_count': 0, 'lik...",1548769294514884610,en,RT @luckytran: Scientists have been warning us...,Twitter for iPhone,everyone,"[{'type': 'retweeted', 'id': '1548380393794969...",50367166,1548380393794969600,3773,NaN
3,1548769288307326976,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",2022-07-17T20:39:07.000Z,"{'retweet_count': 15986, 'reply_count': 0, 'li...",1548769288307326976,en,RT @WhipRealer: It’s a farce. Always has been....,Twitter Web App,everyone,"[{'type': 'retweeted', 'id': '1548315624098566...",1280228272559861761,1548315624098566144,15986,NaN
4,1548769284997906432,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",2022-07-17T20:39:06.000Z,"{'retweet_count': 47, 'reply_count': 0, 'like_...",1548769284997906432,en,RT @Oldglorycries: Climate change is such a th...,Twitter for Android,everyone,"[{'type': 'retweeted', 'id': '1548684629087068...",4481909472,1548684629087068161,47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1548769103003160576,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",2022-07-17T20:38:23.000Z,"{'retweet_count': 10, 'reply_count': 0, 'like_...",1548769103003160576,en,RT @tbas7000: @stephenthegrate @ClimateHuman A...,Twitter for iPhone,everyone,"[{'type': 'retweeted', 'id': '1548111625437163...",332522556,1548111625437163520,10,NaN
92,1548769101899829253,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",2022-07-17T20:38:23.000Z,"{'retweet_count': 105, 'reply_count': 0, 'like...",1548769101899829253,en,RT @nickdixoncomic: ITV presenter just claimed...,Twitter for iPad,everyone,"[{'type': 'retweeted', 'id': '1548741899040612...",800108995365081088,1548741899040612353,105,NaN
93,1548769101224579072,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",2022-07-17T20:38:23.000Z,"{'retweet_count': 7328, 'reply_count': 0, 'lik...",1548769101224579072,en,RT @TheOtherMandela: Joe Manchin just shut dow...,Twitter for Android,everyone,"[{'type': 'retweeted', 'id': '1548079256453885...",756641168188928001,1548079256453885952,7328,NaN
94,1548769099295387648,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",2022-07-17T20:38:22.000Z,"{'retweet_count': 11, 'reply_count': 0, 'like_...",1548769099295387648,en,RT @xr_cambridge: Mark told us: “The devastati...,Twitter for iPhone,everyone,"[{'type': 'retweeted', 'id': '1548594768116154...",1383901595918757891,1548594768116154369,11,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [25]:
climate_feed_df.dtypes

conversation_id          object
entities                 object
created_at               object
public_metrics           object
id                       object
lang                     object
text                     object
source                   object
reply_settings           object
referenced_tweets        object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
in_reply_to_user_id      object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

conversation_id should be an int64
id should be an int64
text should be a string
user_id should be int64
referenced_tweets_ids should be int64

# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [39]:
def tweet_user_generator(tweet_user_list):
    for user in tweet_user_list:
        try:
            yield user
        except KeyError:
            continue

In [40]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [41]:
user_df

,0
0,data
1,includes
2,meta


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [42]:
user_df.dtypes


0    object
dtype: object

# write columns with wrong datatype and mention correct data type too.

0 should be string

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [43]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]

In [44]:
retweeted_status_df.head()

,id,retweet_count
0,1548769297807429632,1110
1,1548769295534014466,284
2,1548769294514884610,3773
3,1548769288307326976,15986
4,1548769284997906432,47


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [47]:
print(sum(retweeted_status_df['retweet_count'] == 0))

11
